In [33]:
## Built-in modules
import os

## Third party modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import OrderedDict
%matplotlib inline

In [60]:
#import biogeme.results as res

In [3]:
#pd.__version__

'1.3.0'

## Process the CFS data

In [44]:
#df_raw = pd.read_csv('cfs_2012_pumf_csv.txt')
#df_raw = pd.read_csv('cfs_2017.csv')

In [45]:
## Only keep the TX Austin data + "Rest of TX"
df_tx = df_raw[(df_raw['ORIG_CFS_AREA'] == '48-12420') | (df_raw['ORIG_CFS_AREA'] == '48-41700') | 
               (df_raw['ORIG_CFS_AREA'] == '48-99999') | (df_raw['DEST_CFS_AREA'] == '48-12420') |  
               (df_raw['DEST_CFS_AREA'] == '48-41700') | (df_raw['DEST_CFS_AREA'] == '48-99999') ]

In [36]:
df_tx.head()

,SHIPMT_ID,ORIG_STATE,ORIG_MA,ORIG_CFS_AREA,DEST_STATE,DEST_MA,DEST_CFS_AREA,NAICS,QUARTER,SCTG,MODE,SHIPMT_VALUE,SHIPMT_WGHT,SHIPMT_DIST_GC,SHIPMT_DIST_ROUTED,TEMP_CNTL_YN,EXPORT_YN,EXPORT_CNTRY,HAZMAT,WGT_FACTOR
24,25,48,99999,48-99999,48,99999,48-99999,311,1,05,4,26498,33269,156,219,Y,N,N,N,114.3
43,44,48,41700,48-41700,48,41700,48-41700,4232,2,43,5,112,116,3,3,N,N,N,N,149.3
113,114,48,41700,48-41700,48,41700,48-41700,4244,2,05,5,94,28,6,8,Y,N,N,N,1667.7
145,146,48,99999,48-99999,48,99999,48-99999,4238,4,40,4,360,36,27,37,N,N,N,N,5080.7
169,170,51,99999,51-99999,48,99999,48-99999,326,4,24,4,22112,28239,1042,1290,N,N,N,N,106.6


In [51]:
df_tx.shape

(253810, 22)

In [49]:
#df_tx['MODE'].value_counts()

### Generate mode choice variables

In [ ]:
#df_tx.loc[df_tx['MODE'] == 4, 'mode_agg5'] = 'For-hire Truck'

In [46]:
mode_agg5_dict = {4:'For-hire Truck', 
                  5:'Private Truck', 
                  6:'Rail/IMX', 
                  15:'Rail/IMX', 
                  11:'Air', 
                  14:'Parcel', 
                  0:'Other',
                  2:'Other',
                  3:'Other',
                  7:'Other',
                  8:'Other',
                  10:'Other',
                  12:'Other',
                  13:'Other',
                  16:'Other',
                  17:'Other',
                  20:'Other',
                  101:'Other'}

mode_agg3_dict = {3:'Truck',
                  4:'Truck', 
                  5:'Truck', 
                  6:'Rail/IMX', 
                  15:'Rail/IMX', 
                  11:'Air/Parcel', 
                  14:'Air/Parcel', 
                  0:'Other',
                  2:'Other',
                  7:'Other',
                  8:'Other',
                  10:'Other',
                  12:'Other',
                  13:'Other',
                  16:'Other',
                  17:'Other',
                  20:'Other',
                  101:'Other'}

df_tx['mode_agg5'] = (df_tx.MODE).replace(mode_agg5_dict).copy()
df_tx['mode_agg3'] = (df_tx.MODE).replace(mode_agg3_dict).copy()

In [9]:
#df_tx[['MODE','mode_agg3','mode_agg5']].head(20)

In [53]:
#df_tx['mode_agg5'].value_counts(dropna=False)

In [54]:
#df_tx.groupby(['mode_agg5'])['WGT_FACTOR'].agg('sum')

In [11]:
## Remove shipment with mode other than the 5 modes

df_tx = df_tx[df_tx['mode_agg5'] != 'Other'] 

In [43]:
df_tx['mode_agg5'].value_counts()

For-hire Truck    72799
Parcel            47848
Private Truck     47575
Rail/IMX           4197
Air                2386
Other               590
Name: mode_agg5, dtype: int64

### Generate commodity type variables

In [13]:
df_tx['SCTG'] = df_tx['SCTG'].astype(int)

In [14]:
df_tx['bulk'] = np.where((df_tx['SCTG'] == 2) | (df_tx['SCTG'] == 10) | (df_tx['SCTG'] == 11) | 
                         (df_tx['SCTG'] == 12) | (df_tx['SCTG'] == 13) | (df_tx['SCTG'] == 14) | 
                         (df_tx['SCTG'] == 15) | (df_tx['SCTG'] == 25) | (df_tx['SCTG'] == 26)| (df_tx['SCTG'] == 32), 1, 0)
df_tx['fuel_fert'] = np.where((df_tx['SCTG'] == 16) | (df_tx['SCTG'] == 17) | (df_tx['SCTG'] == 18) | 
                              (df_tx['SCTG'] == 19) | (df_tx['SCTG'] == 20) | (df_tx['SCTG'] == 22) | (df_tx['SCTG'] == 23), 1, 0)
df_tx['interm_food'] = np.where((df_tx['SCTG'] == 1) | (df_tx['SCTG'] == 3) | (df_tx['SCTG'] == 4) | 
                         (df_tx['SCTG'] == 5) | (df_tx['SCTG'] == 6) | (df_tx['SCTG'] == 7) | (df_tx['SCTG'] == 8), 1, 0)
df_tx['mfr_goods'] = np.where((df_tx['SCTG'] == 9) | (df_tx['SCTG'] == 21) | (df_tx['SCTG'] == 24) | 
                              (df_tx['SCTG'] == 27) | (df_tx['SCTG'] == 28) | (df_tx['SCTG'] == 29) | 
                              (df_tx['SCTG'] == 30) | (df_tx['SCTG'] == 31) | (df_tx['SCTG'] == 33) | 
                              (df_tx['SCTG'] == 34) | (df_tx['SCTG'] == 35) | (df_tx['SCTG'] == 36) |
                              (df_tx['SCTG'] == 37) | (df_tx['SCTG'] == 38) | (df_tx['SCTG'] == 39) | (df_tx['SCTG'] == 40), 1, 0)
df_tx['other'] = np.where((df_tx['SCTG'] == 41) | (df_tx['SCTG'] == 43) | (df_tx['SCTG'] == 99), 1, 0)

## pd.get_dummies() - https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
## long_swiss_metro["single_luggage_piece"] = (long_swiss_metro["LUGGAGE"] == 1).astype(int)
## df_long['hot_climate'] = (df_long['climate_zone'].isin(['Hot-Dry', 'Hot-Humid'])).astype(int)

In [16]:
#df_tx[['bulk', 'fuel_fert', 'interm_food', 'mfr_goods','other']]

In [17]:
commodity_dict = {2:'bulk', 10:'bulk', 11:'bulk', 12:'bulk', 13:'bulk', 14:'bulk', 15:'bulk', 25:'bulk', 26:'bulk', 32:'bulk',
                  16:'fuel_fert', 17:'fuel_fert', 18:'fuel_fert', 19:'fuel_fert', 20:'fuel_fert', 22:'fuel_fert', 23:'fuel_fert',
                  1:'interm_food', 3:'interm_food', 4:'interm_food', 5:'interm_food', 6:'interm_food', 7:'interm_food', 8:'interm_food',
                  9:'mfr_goods', 21:'mfr_goods', 24:'mfr_goods', 27:'mfr_goods', 28:'mfr_goods', 29:'mfr_goods', 30:'mfr_goods', 
                  31:'mfr_goods', 33:'mfr_goods', 34:'mfr_goods', 35:'mfr_goods', 36:'mfr_goods', 37:'mfr_goods', 38:'mfr_goods', 
                  39:'mfr_goods', 40:'mfr_goods',
                  41:'other', 43:'other', 99:'other'}

df_tx['commodity'] = df_tx['SCTG'].map(commodity_dict)

#df_tx['commodity'] = (df_tx.SCTG).replace(commodity_dict)

In [18]:
#df_tx['commodity'].value_counts(dropna=False)

In [19]:
#df_tx.groupby(['mode_agg5','commodity'])['WGT_FACTOR'].agg('count')

### Generate NAICS industry type variables

In [20]:
df_tx['naics2'] = df_tx['NAICS'].astype(str).str[:2].astype(int)

In [21]:
df_tx['naics_name'] = (df_tx.naics2).replace({21:'Mining', 
                                              31:'Manufacturing',
                                              32:'Manufacturing',
                                              33:'Manufacturing',
                                              42:'Wholesale',
                                              45:'Retail',
                                              49:'Trans_Warehouse',
                                              51:'Information',
                                              55:'Mgt_companies'})

df_tx['naics_name'].value_counts() 

Manufacturing      89647
Wholesale          65568
Retail              5973
Mining              5875
Trans_Warehouse     4441
Information         2080
Mgt_companies       1221
Name: naics_name, dtype: int64

In [22]:
#df_tx.groupby(['mode_agg5','naics_name'])['WGT_FACTOR'].agg('count')

In [23]:
#df_tx.groupby(['mode_agg5','naics_name'])['WGT_FACTOR'].agg('count')

In [24]:
df_tx["wholesale"] = (df_tx['naics_name'] == "Wholesale").astype(int)
df_tx["mfring"] = (df_tx['naics_name'] == "Manufacturing").astype(int)
df_tx["mining"] = (df_tx['naics_name'] == "Mining").astype(int)
df_tx["retail"] = (df_tx['naics_name'] == "Retail").astype(int)
df_tx["info"] = (df_tx['naics_name'] == "Information").astype(int)
df_tx["management"] = (df_tx['naics_name'] == "Mgt_companies").astype(int)
df_tx["transwarehouse"] = (df_tx['naics_name'] == "Trans_Warehouse").astype(int)

### Generate shipment weight bins

In [25]:
df_tx['wght_bin1'] = np.where(df_tx['SHIPMT_WGHT'] <= 150, 1,
                     np.where((df_tx['SHIPMT_WGHT'] > 150) & (df_tx['SHIPMT_WGHT'] <= 1500), 2,
                     np.where((df_tx['SHIPMT_WGHT'] > 1500) & (df_tx['SHIPMT_WGHT'] <= 30000), 3,
                     np.where((df_tx['SHIPMT_WGHT'] > 30000) & (df_tx['SHIPMT_WGHT'] <= 45000), 4, 5))))

#df_tx['wght_bin2'] = np.where(df_tx['SHIPMT_WGHT'] <= 150, 1,
#                     np.where((df_tx['SHIPMT_WGHT'] > 150) & (df_tx['SHIPMT_WGHT'] <= 1000), 2,
#                     np.where((df_tx['SHIPMT_WGHT'] > 1000) & (df_tx['SHIPMT_WGHT'] <= 5000), 3,
#                     np.where((df_tx['SHIPMT_WGHT'] > 5000) & (df_tx['SHIPMT_WGHT'] <= 35000), 4, 5))))

### Generate additional variables

In [26]:
df_tx['geo'] = np.where(((df_tx['ORIG_CFS_AREA'] == '48-12420') | (df_tx['ORIG_CFS_AREA'] == '48-41700')) & 
               ((df_tx['DEST_CFS_AREA'] == '48-12420') | (df_tx['DEST_CFS_AREA'] == '48-41700')), 'Within Austin', 'External')

df_tx['SHIPMT_WGHT_TON'] = df_tx['SHIPMT_WGHT']/2000

df_tx['value_density'] = df_tx['SHIPMT_VALUE']/df_tx['SHIPMT_WGHT']

df_tx['SHIPMT_DIST'] = df_tx['SHIPMT_DIST_ROUTED']
df_tx['SHIPMT_DIST'] = np.where((df_tx['mode_agg5'] == 'Air') | (df_tx['mode_agg5'] == 'Parcel'), 
                                df_tx['SHIPMT_DIST_GC'],df_tx['SHIPMT_DIST']) # Use GC distance for Air and Parcel and Routed distance for the rest of modes.

### Additional data removal rules

In [27]:
#df_tx.groupby(['mode_agg5'])['value_density'].describe()

In [28]:
#df_tx.groupby(['commodity'])['value_density'].describe()

In [29]:
## based on Stinson et al. (2017)

df_tx.drop(df_tx[(df_tx['mode_agg5'] == 'Air') & (df_tx['SHIPMT_WGHT'] > 15000)].index, inplace = True)
df_tx.drop(df_tx[(df_tx['mode_agg5'] == 'Air') & (df_tx['SHIPMT_WGHT'] > 150) & (df_tx['value_density'] < 1)].index, inplace = True)
df_tx.drop(df_tx[(df_tx['mode_agg5'] == 'Parcel') & (df_tx['SHIPMT_WGHT'] > 150) & (df_tx['value_density'] < 1)].index, inplace = True)
df_tx.drop(df_tx[(df_tx['MODE'] == 6) & (df_tx['SHIPMT_WGHT'] < 1500)].index, inplace = True)
df_tx.drop(df_tx[(df_tx['MODE'] == 6) & (df_tx['value_density'] >= 4)].index, inplace = True)


In [30]:
df_tx.shape

(174715, 43)